In [2]:
import numpy as np
from itertools import combinations

def held_karp(distance_matrix):
    n = len(distance_matrix)
    C = {}
    
    # 初始化：从起点到其他城市的单步距离
    for k in range(1, n):
        C[(1 << k, k)] = (distance_matrix[0][k], 0)
    
    # 遍历所有子集大小
    for subset_size in range(2, n):
        for subset in combinations(range(1, n), subset_size):
            bits = 0
            for bit in subset:
                bits |= 1 << bit
            
            # 对每个子集中的城市 k，找到最短路径
            for k in subset:
                prev_bits = bits & ~(1 << k)
                min_dist = np.inf
                min_prev = 0
                for m in subset:
                    if m == k:
                        continue
                    dist, _ = C.get((prev_bits, m), (np.inf, 0))
                    dist += distance_matrix[m][k]
                    if dist < min_dist:
                        min_dist = dist
                        min_prev = m
                C[(bits, k)] = (min_dist, min_prev)
    
    # 计算回到起点的总距离
    bits = (1 << n) - 2  # 所有位为1（除了起点0）
    min_total = np.inf
    last = 0
    for k in range(1, n):
        dist, prev = C.get((bits, k), (np.inf, 0))
        dist += distance_matrix[k][0]
        if dist < min_total:
            min_total = dist
            last = k
    
    # 回溯路径
    path = []
    bits = (1 << n) - 2
    while last != 0:
        path.append(last)
        new_bits = bits & ~(1 << last)
        _, prev = C[(bits, last)]
        bits = new_bits
        last = prev
    path.append(0)
    path.reverse()
    return min_total, [0] + path

# 示例使用（5个城市）
n = 5
np.random.seed(42)
points = np.random.rand(n, 2)
d = np.zeros((n, n))
for i in range(n):
    for j in range(n):
        d[i][j] = np.linalg.norm(points[i] - points[j])

min_dist, path = held_karp(d)
print("最优路径:", path)
print("总距离:", min_dist)

最优路径: [0, 0, 4, 1, 2, 3]
总距离: 2.2734551867849095


In [3]:
import numpy as np
import random

def simulated_annealing(distance_matrix, max_iter=10000, initial_temp=1000, cooling_rate=0.995):
    n = len(distance_matrix)
    current_path = list(range(n))
    random.shuffle(current_path)
    current_cost = calculate_cost(current_path, distance_matrix)
    
    best_path = current_path.copy()
    best_cost = current_cost
    
    temp = initial_temp
    
    for _ in range(max_iter):
        # 生成新路径（交换两个城市）
        new_path = current_path.copy()
        i, j = random.sample(range(n), 2)
        new_path[i], new_path[j] = new_path[j], new_path[i]
        new_cost = calculate_cost(new_path, distance_matrix)
        
        # 计算成本差
        delta = new_cost - current_cost
        if delta < 0 or np.exp(-delta / temp) > random.random():
            current_path = new_path
            current_cost = new_cost
        
        # 更新最优解
        if current_cost < best_cost:
            best_path = current_path.copy()
            best_cost = current_cost
        
        # 降温
        temp *= cooling_rate
    
    # 闭合路径（返回起点）
    best_path.append(best_path[0])
    return best_cost, best_path

def calculate_cost(path, distance_matrix):
    cost = 0
    for i in range(len(path)-1):
        cost += distance_matrix[path[i]][path[i+1]]
    return cost

# 示例使用（20个城市）
n = 20
np.random.seed(42)
points = np.random.rand(n, 2)
d = np.zeros((n, n))
for i in range(n):
    for j in range(n):
        d[i][j] = np.linalg.norm(points[i] - points[j])

best_cost, best_path = simulated_annealing(d)
print("近似最优路径:", best_path)
print("总距离:", best_cost)

近似最优路径: [14, 9, 8, 11, 7, 2, 18, 1, 17, 6, 10, 15, 19, 4, 0, 16, 5, 3, 13, 12, 14]
总距离: 5.089692491312108


In [4]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

def solve_tsp_ortools(distance_matrix):
    n = len(distance_matrix)
    manager = pywrapcp.RoutingIndexManager(n, 1, 0)  # 1辆车，起点为0
    routing = pywrapcp.RoutingModel(manager)
    
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return distance_matrix[from_node][to_node]
    
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    search_parameters.time_limit.seconds = 30
    
    solution = routing.SolveWithParameters(search_parameters)
    
    if solution:
        path = []
        index = routing.Start(0)
        while not routing.IsEnd(index):
            path.append(manager.IndexToNode(index))
            index = solution.Value(routing.NextVar(index))
        path.append(0)  # 返回起点
        return solution.ObjectiveValue(), path
    else:
        return None, None

# 示例使用（10个城市）
n = 10
np.random.seed(42)
points = np.random.rand(n, 2)
d = np.zeros((n, n))
for i in range(n):
    for j in range(n):
        d[i][j] = np.linalg.norm(points[i] - points[j])

min_dist, path = solve_tsp_ortools(d)
print("最优路径:", path)
print("总距离:", min_dist)

load d:\Python3.12\Lib\site-packages\ortools\.libs\zlib1.dll...
load d:\Python3.12\Lib\site-packages\ortools\.libs\abseil_dll.dll...
load d:\Python3.12\Lib\site-packages\ortools\.libs\utf8_validity.dll...
load d:\Python3.12\Lib\site-packages\ortools\.libs\re2.dll...
load d:\Python3.12\Lib\site-packages\ortools\.libs\libprotobuf.dll...
load d:\Python3.12\Lib\site-packages\ortools\.libs\highs.dll...
load d:\Python3.12\Lib\site-packages\ortools\.libs\ortools.dll...
最优路径: [0, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0]
总距离: 0
